In [6]:
import json
import pandas as pd
import numpy as np
import folium
import osmnx as ox # installed via pip
import geopy
from itertools import product
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import openrouteservice as ors  # installed via Terminal since the channel isn´t in the navigator
from openrouteservice.directions import directions
import time
import csv
#from methods import *
import geopandas as gpd
import random
import json
from shapely import Point, Polygon
from scipy.spatial import ConvexHull
import math
import openpyxl
import copy


locator = geopy.Nominatim(user_agent='TUM_Masterthesis_Hohmann')
client = ors.Client(key='5b3ce3597851110001cf62482ac01804d2d6434a8cdd87785bfb541a')  # Specify your personal API key
random.seed(23)

In [69]:
### --- Definition of functions --- ###
def coord_min_to_decimal(coord_str):
    coord_list = coord_str.split(",")
    coord_dec = int(coord_list[0]) + int(coord_list[1])/60 + int(coord_list[2])/3600
    return coord_dec

def set_style(color):
    # function to style data for the displaying of the solution
    return lambda feature: dict(color=color,
                                opacity=0.5,
                                weight=4, )

def pythagoras(_start, _end):
    # calculate the pythagoras distance between two nodes, both inputs should be Coord
    _diff_lon = _start.lon - _end.lon
    _diff_lat = _start.lat - _end.lat
    _dist = math.sqrt(_diff_lon ** 2 + _diff_lat ** 2)
    return round(_dist, 2)

def dijkstra(_distance, _starting_node):
    # dijkstra algorythm to find the shortest path from one starting node to all other nodes
    # initialize
    _new_distance = {v: math.inf for v in V}
    _predecessor = {v: None for v in V}
    _new_distance[_starting_node] = 0
    _unvisited_vertices = V.copy()
    # algorythm
    while (len(_unvisited_vertices)) > 0:
        _help_dist = {v: _new_distance[v] for v in _unvisited_vertices}
        _i = min(_help_dist, key=_help_dist.get)  # node with minimal distance as current node
        del _unvisited_vertices[i]  # delete node from unvisited nodes
        for _j in V:
            if (_i, _j) in E:  # all neighbors
                if _j in _unvisited_vertices:
                    _alt_dist = _new_distance[i] + _distance[i][j]
                    if _alt_dist < _new_distance[j]:
                        _new_distance[j] = round(_alt_dist, 2)
                        _predecessor[j] = i
    return _new_distance, _predecessor

def calc_angle(vector1, vector2):
    # vector needs to be a np.array
    vector1_norm = vector1 / np.linalg.norm(vector1)  # normalizes vector
    vector2_norm = vector2 / np.linalg.norm(vector2)
    # np.dot: dot product of two vectors; np.clip: minimal -1, maximal, 1
    return 180 / np.pi * np.arccos(np.clip(np.dot(vector1_norm, vector2_norm), -1.0, 1.0))

def calc_driving_time(_distance):
    # calculates the driving time from node i to node j in time steps
    driving_time = {i: {j: math.ceil(_distance[i][j] / Velocity / Len_time_step)
                        for j in _distance[i]} for i in _distance}
    return driving_time

def find_exact_path(_start, _end, _predecessor_dict):
    # returns the path of nodes taken to get from start to end customer, no other customers in between
    _path = [_end]
    while _end != _start:
        _end = _predecessor_dict[_start][_end]
        _path.insert(0, _end)
    return _path

def find_start_index_of_arc(_route, _start, _end):
    _ind = R[_route].exact_tour.index(_start)
    while _end is not R[route].exact_tour[_ind + 1]:
        _ind = R[_route].exact_tour.index(_start, _ind + 1)
    return _ind

def include_path_into_tour(_route, _start, _end):
    # check if any arc is not yet in exact_tour
    # customers can only be visited once by a tour! otherwise we would have a problem here
    if _start not in _route.exact_tour and _end not in _route.exact_tour:
        return f'ERROR: arc {_start, _end} is not on tour {_route.ID}!'
    elif _start in _route.exact_tour and _end in _route.exact_tour:
        _ind_start = _route.exact_tour.index(_start)
        _ind_end = _route.exact_tour.index(_end)
        del _route.exact_tour[_ind_start + 1, _ind_end]  # remove all nodes that are in between start and end
        _ind_end = _ind_start + 1
    elif _start not in _route.exact_tour and _end in _route.exact_tour:
        _ind_start = _route.exact_tour.index(_end)  # index of start is the current index of end
        # because end is shifted once to the right
        _route.exact_tour.insert(_ind_start, _start)
        _ind_end = ind_start + 1  # probably faster than finding the index again
    elif _start in _route.exact_tour and _end not in _route.exact_tour:
        _ind_start = _route.exact_tour.index(_start)
        _ind_end = _ind_start + 1
        _route.exact_tour.insert(ind_end, _end)
    else:
        return f'no idea how i reached this point'

    # now all three cases have the same situation: start and end are next to each other
    _path = path_fwa[_start][_end][1:-1]  # shortest path excluded the start and end
    _new_tour = _route.exact_tour
    for n in _path:
        _new_tour.insert(_ind_end, n)
    return _new_tour

def create_platoons():
    for (i, j) in E:  # reset possible platoons
        Platoon_arcs[i][j] = []
    # find arcs, that are travelled by several trucks
    for r in R.values():
        for i in range(len(r.exact_tour) - 1):
            Platoon_arcs[r.exact_tour[i]][r.exact_tour[i + 1]].append(r.ID)

    for r in R.values():
        r.platoon = [0] * (len(r.exact_tour) - 1)
        r.update_times()

    for _start, _end_val in Platoon_arcs.items():
        for _end, _route_ids in _end_val.items():
            print(_start, _end, _route_ids)
            if len(_route_ids) > 1:
                start_index = {r: find_start_index_of_arc(r, _start, _end) for r in _route_ids}
                leaving_times = {r: R[r].leaving_times[start_index[r]] for r in _route_ids}
                latest_route = max(leaving_times, key=leaving_times.get)
                for other_route in _route_ids:
                    if other_route is not latest_route:
                        time_diff = leaving_times[latest_route] - leaving_times[other_route]
                        if time_diff >= 0:  # if time difference is equal 0, both tours can already form a platoon
                            R[latest_route].platoon[start_index[latest_route]] = 1
                            R[other_route].platoon[start_index[other_route]] = 1
                            if time_diff > 0:
                                # if time diff is greater 0, the leaving times of the earlier tour have to be adjusted
                                # rest_route = R[other_route].exact_tour[start_index[other_route]:]
                                for index_later_nodes in \
                                        range(len(R[other_route].exact_tour[start_index[other_route]:])):
                                    # iterate over the nodes, that are later than the start node of the platooning arc
                                    # the first node to be updated is the start node (index_late_nodes = 0)
                                    R[other_route].leaving_times[
                                        start_index[other_route] + index_later_nodes] += time_diff
                                # routes[latest_route].driving_time = routes[latest_route].leaving_times[-1]
                                # routes[other_route].driving_time = routes[other_route].leaving_times[-1]
                        else:
                            print(
                                'Here is a problem: Time difference between the latest route '
                                'and another should be greater or equal 0!')
                del other_route
        del _end, _route_ids
    del _start, _end_val

def create_gene_from_tours():
    _gene_code = []
    _depots = []
    for r in R.values():
        for i in r.main_tour[1:-1]:
            _gene_code.append(i)
            _depots.append(r.main_tour[0])
        del i
    del r
    _gene = Genecode(gene=_gene_code, depot_allocation=_depots)
    return _gene

# functions for GA
def find_borderline_alternative_depot(_customer, _closest_depot):
    _alternative_depots = []
    dist_to_closest_depot = dist_fwa[_customer][_closest_depot]
    for _alt_depot in D:
        if _alt_depot is not _closest_depot:
            dist_to_alt = dist_fwa[_customer][_alt_depot]
            if dist_to_alt <= Borderline_factor * dist_to_closest_depot:
                _alternative_depots.append(_alt_depot)
    return _alternative_depots

def create_chromosome(_individual):
    _chromosome = {d: [] for d in D}
    for _r in _individual:
        _route_depot = _r[0]
        for _c in _r[1:-1]:  # first and last stop is depot
            _chromosome[_route_depot].append(_c)
    return _chromosome

def route_scheduler(_chromosome):
    _route_list = []
    # has to return dict of routes, good idea? or better leave out the class route
    for _depot, _sequence in _chromosome.items():
        _routes_depot = []
        _route_demand = 0
        _route_drivetime = 0
        _route_worktime = 0
        _route = [_depot, _depot]
        for c in _sequence:
            if check_route_conditions(_route, c, _route_demand, _route_drivetime, _route_worktime):
                # insert new customer to route and update route parameters
                _route_demand += V[c].demand
                _diff_drivetime = drive_time_fwa[_route[-2]][c] + drive_time_fwa[c][_route[-1]] - drive_time_fwa[_route[-2]][_route[-1]]
                _route_drivetime +=  _diff_drivetime
                _route_worktime = _route_worktime + _diff_drivetime + V[c].service_time
                _route.insert(-1, c)
                if c == _sequence[-1]:
                    _routes_depot.append(_route)
            else:
                # start new route
                _routes_depot.append(_route) # save finished route

                _route = [_depot, c, _depot]
                _route_demand = V[c].demand
                _route_drivetime = drive_time_fwa[_depot][c] + drive_time_fwa[c][_depot]
                _route_worktime  = _route_drivetime + V[c].service_time
        if len(_routes_depot) > 1: # more than one route has been updated, check if nodes can be shifted.
            _best_fitness = evaluate_fitness_of_individual(_routes_depot)
            for i in range(-2, - len(_routes_depot) - 1, -1):
                _demand, _drivetime, _worktime = calc_times_demand(_routes_depot[i+1])  # calc current requirement values of route where the customer should shift INTO
                _shifting_customer = _routes_depot[i][-2] # last customer ([-2] because [-1] is depot) of second last route shifts into last route
                if check_route_conditions(_routes_depot[i+1],_shifting_customer, _demand, _drivetime, _worktime):
                    _shifted_routes = copy.deepcopy(_routes_depot)
                    _shifted_routes[i].remove(_shifting_customer) # remove customer from old route
                    _shifted_routes[i+1].insert(1, _shifting_customer) # insert customer to new route
                    _shifted_fitness = evaluate_fitness_of_individual(_shifted_routes)
                    if _shifted_fitness <= _best_fitness:
                        _best_fitness = _shifted_fitness
                        _routes_depot = _shifted_routes

        _route_list = _route_list + _routes_depot
    return _route_list

def evaluate_fitness_of_individual(_individual):
    # input is a list of routes, fitness counts for the entire genome
    _fitness = Cost_factor_depreciation * len(_individual)  # first cost factor is independent of all touring and scheduling

    _exact_route_dict = {i: [_individual[i][0]] for i in range(len(_individual))} # dict so route can be found more easily again
    # find exact routes
    for _r_ind in range(len(_individual)):
        _route = _individual[_r_ind]
        _current_exact_route = [_route[0]]
        for _node in _route[1:]:
            _newPath = find_exact_path(_current_exact_route[-1], _node, pred_fwa)
            _current_exact_route = _current_exact_route + _newPath[1:]
        _exact_route_dict[_r_ind]=_current_exact_route
    # earliest times
    _earliest_times = {k: {n: None for n in _exact_route_dict[k]} for k in _exact_route_dict}
    # depot is only the second time in there, its obvious that the earliest leaving time at the depot is 0.
    for _key, _route in _exact_route_dict.items():
        for _n_ind in range(1, len(_route)):

            if _n_ind == 1: # leaving the depot
                _earliest_times[_key][_n_ind] = V[_route[_n_ind-1]].service_time + drive_time_fwa[_route[_n_ind-1]][_route[_n_ind]] + V[_route[_n_ind]].service_time
            else:
                _earliest_times[_key][_n_ind] = _earliest_times[_key][_n_ind-1] + drive_time_fwa[_route[_n_ind-1]][_route[_n_ind]] + V[_route[_n_ind]].service_time






    # find arcs where more than one truck is driving
    _platoon_arcs = {(i, j): [] for (i, j) in E}

    for _key, _route in _exact_route_dict.items():
        for i in range(len(_route)-1):
            _platoon_arcs[(_route[i],_route[i+1])].append(_key)





    '''for _route in _individual:
        _fitness += evaluate_fitness_of_route(_route)'''
    #
    return  _fitness

def evaluate_fitness_of_route(_route):
    _fitness = 0
    _cost_labor = Cost_factor_labor * V[_route[0]].service_time
    _cost_fuel = 0
    for _start_ind in range(1, len(_route)):
        _cost_fuel += Cost_factor_fuel * dist_fwa[_route[_start_ind- 1]][_route[_start_ind]]
        _cost_labor += Cost_factor_labor * ( drive_time_fwa[_route[_start_ind- 1]][_route[_start_ind]] + V[_route[_start_ind]].service_time )
    _fitness += round(_cost_labor + _cost_fuel,3)
    return _fitness

def calc_times_demand(_route):
    _demand = V[_route[0]].demand
    _worktime = V[_route[0]].service_time
    _drivetime = 0
    for _s in range(1, len(_route)):
        _drivetime += drive_time_fwa[_route[_s-1]][_route[_s]]
        _worktime += drive_time_fwa[_route[_s-1]][_route[_s]] + V[_route[_s]].service_time
        _demand += V[_route[_s]].demand
    return _demand, _drivetime, _worktime

def check_route_conditions(_route, _new_customer, _demand, _drivetime, _worktime):
    """Route input has to be a list."""
    # check new demand
    if _demand + V[_new_customer].demand <= Capacity_truck:
        # check work time
        _new_drivetime = _drivetime - drive_time_fwa[_route[-2]][_route[-1]] + drive_time_fwa[_route[-2]][_new_customer] + drive_time_fwa[_new_customer][_route[-1]]
        if _new_drivetime <= Max_driving_time:
            _new_worktime = _worktime - drive_time_fwa[route[-2]][route[-1]] + drive_time_fwa[_route[-2]][_new_customer] + drive_time_fwa[_new_customer][_route[-1]] + V[_new_customer].service_time
            if _new_worktime <= Max_working_time:
                return True
            else:
                return False
        else:
            return False
    else:
        return False

def crossover_order(_chromosome1, _chromosome2):
    # chromosome1 is the main parent, who dictates length and customers of child
    _child_chromosome = dict()
    for _d in _chromosome1:
        _p1 = _chromosome1[_d]
        _p2 = _chromosome2[_d]
        if len(_p1) < 3:  # cannot cut into three pieces if only two elements
            return _p1
        _child = [0] * len(_p1)
        _ind_list = list(range(len(_p1)))
        _cut1 = random.randint(1,len(_p1)-1)
        _cut2 = _cut1
        while _cut2 == _cut1:
            _cut2 = random.randint(1, len(_p1)-1)
        _c1 = min(_cut1, _cut2)  # we need to know which cut comes first
        _c2 = max(_cut1, _cut2)
        _child[_c1: _c2] = _p1[_c1: _c2]
        _rest = [i for i in (_p2[_c2:] + _p2[:_c2]) if i in _p1] # kicks out all nodes in _p2 which are not in _p1
        _rest = _rest + [i for i in _p1 if i not in _p2] # adds all nodes to rest which are only in _p1
        _empty_ind = _ind_list[_c2:] + _ind_list[:_c1]
        for _ind in _empty_ind:
            while _rest[0] in _child:
                _rest.remove(_rest[0])
            _child[_ind] = _rest[0]
        _child_chromosome[_d] = _child

    return _child_chromosome # return only one child, so each generation produces one single offspring

def crossover_alternating_edges(_chromosome1, _chromosome2):
    _child_chromosome = dict()
    for _d in _chromosome1:
        _p1 = _chromosome1[_d]
        _p2 = _chromosome2[_d]
        _child = _p1[:2]
        _next_parent = _p2
        _error = False
        while len(_child) < len(_p1):
            _start = _child[-1]
            _ind = _next_parent.index(_start)
            if _ind + 1 == len(_next_parent): # edge case if the last arc is the same for both parents
                _error = True
                _end = None
            else:
                _end = _next_parent[_ind + 1]
            if _end not in _p1:
                _error = True
            if _end in _child:
                _error = True
            if _error:
                _rest = [i for i in _p1 if i not in _child]
                _end = random.choice(_rest)
            _child.append(_end)
            if _next_parent == _p2:
                _next_parent = _p1
            else:
                _next_parent = _p2
        _child_chromosome[_d] = _child
    return _child_chromosome

def mutation_swap(_chromosome):
    # accepts a gene (list of nodes) as input
    if len(_chromosome) <2:
        return _chromosome
    _ind1 = random.choice(range(len(_chromosome)))
    _ind2 = _ind1
    while _ind2 == _ind1:
        _ind2 = random.choice(range(len(_chromosome)))

    _node1 = _chromosome[_ind1]
    _node2 = _chromosome[_ind2]

    _chromosome[_ind1] = _node2
    _chromosome[_ind2] = _node1

    return _chromosome

def mutation_inversion(_chromosome):
    # accepts a gene (list of nodes) as input
    if len(_chromosome) <2:
        return _chromosome
    _ind1 = random.choice(range(len(_chromosome)+1))
    _ind2 = _ind1
    while _ind2 == _ind1:
        _ind2 = random.choice(range(len(_chromosome)+1))
    _i1 = min(_ind1, _ind2)
    _i2 = max(_ind1, _ind2)

    _part1 = _chromosome[:_i1]
    _middle = _chromosome[_i1:_i2]
    _part2 = _chromosome[_i2:]
    _middle.reverse()
    _chromosome = _part1 + _middle + _part2
    return _chromosome

def mutation_depot_switch(_individual):
    #list of depots
    _depots = list(_individual.keys()).copy()
    while len(_depots)>0:  # if len == 0, no nodes with alternative depots found, original individual is returned
        _d = random.choice(_depots) # random depot
        # search for customers at this depot, that has alternative depots
        _customers_with_alt_depots = []
        for _n in _individual[_d]:
            if len(alternative_depot[_n]) > 0:
                _customers_with_alt_depots.append(_n)
        if len(_customers_with_alt_depots) == 0:
            # delete current depot and choose another one
            _depots.remove(_d)
        else:
            # continue algorithm
            _current_depot = _d
            _node = random.choice(_customers_with_alt_depots)
            _new_depot = random.choice(alternative_depot[_node])
            alternative_depot[_node].remove(_new_depot)
            alternative_depot[_node].append(_current_depot)
            _individual[_d].remove(_node)
            _insertion_ind = random.randint(0,len(_individual))
            _individual[_new_depot].insert(_insertion_ind, _node)
            return _individual
    return _individual

def tournament_selection(_population, _fitness):
    _weights = [Tournament_probability* (1-Tournament_probability)**i for i in range(Tournament_size-1)]
    _weights.append(1- sum(_weights))
    _participants = random.sample(list(_population.keys()), Tournament_size)
    _fitness_participants = {p: _fitness[p]  for p in _participants} # new fitness dict with only the participants
    _sorted_participants = dict(sorted(_fitness_participants.items(), key=lambda item: item[1]))
    _sorted_participants = list(_sorted_participants.keys())
    _parent1 = random.choices(_sorted_participants, _weights, k=1)[0]
    _parent2 = _parent1
    while _parent2 == _parent1:
        _participants = random.sample(list(_population.keys()), Tournament_size)
        _fitness_participants = {p: _fitness[p]  for p in _participants} # new fitness dict with only the participants
        _sorted_participants = dict(sorted(_fitness_participants.items(), key=lambda item: item[1]))
        _sorted_participants = list(_sorted_participants.keys())
        _parent2 = random.choices(_sorted_participants, _weights, k=1)[0]

    return _parent1, _parent2


def survivor_selection(_population):
    while len(_population) > Population_size:
        _fitness = {k: fitness[k] for k in _population}
        _sorted_keys = sorted(_fitness.keys(), key=lambda key: _fitness[key], reverse=True)
        _weights = [Survivor_probability * (1 - Survivor_probability)**i for i in range(len(_sorted_keys))]
        _weights[-1] = 0
        _victim = random.choices(_sorted_keys, _weights, k=1)[0]
        del _population[_victim]
    return _population


# functions for ORM
def distance_ors(_start_lon, _start_lat, _end_lon, _end_lat):

    _coordinates = ((_start_lon, _start_lat), (_end_lon, _end_lat))
    _direction_params = {'coordinates': _coordinates,  # lon, lat
                        'profile': 'driving-hgv',
                        'format_out': 'geojson',
                        'preference': 'shortest',
                        'geometry': 'true'}
    _route = directions(client=client, **_direction_params)
    time.sleep(1.6)  # wait for a short time because the server only allows 40 accesses per minute
    _distance, _duration = _route['features'][0]['properties']['summary'].values()
    return _route, _distance

def generate_demand():
    _demand = np.random.normal(loc=70, scale=20, size=1)
    if _demand < 30:
        _demand = 30
    elif _demand > 110:
        _demand = 110
    return round(_demand[0],0)

def plot_routes(_routes, _map_name):
    _loc = ox.geocode('NRW, Germany')
    _map = folium.Map(location=_loc, start_zoom=11)
    #plot customers
    for c in C.values():
        _text = c.ID
        _marker = folium.Marker(
            location=(c.coord.lat, c.coord.lon),  # Set the marker's latitude and longitude
            popup=folium.Popup(_text, parse_html=True),  # Set the text content inside a popup
            icon=folium.DivIcon(
            icon_size=(30, 30),  # Adjust the size of the circle as needed
            icon_anchor=(15, 15),  # Position the circle's center
            html=f'<div style="font-size: 10pt; text-align: center;">' +
                 f'<svg width="20" height="20">' +
                 f'<circle cx="10" cy="10" r="9" fill="red" />' +
                 f'<text x="10" y="12" fill="white" text-anchor="middle">C</text>' +
                 f'</svg>' +
                 f'</div>'
            ),
        )
        _marker.add_to(_map)
    #plot depots
    for d in D.values():
        _text = d.ID
        _marker = folium.Marker(
            location=(d.coord.lat, d.coord.lon),  # Set the marker's latitude and longitude
            popup=folium.Popup(_text, parse_html=True),  # Set the text content inside a popup
            icon=folium.DivIcon(
                icon_size=(30, 30),  # Adjust the size of the circle as needed
                icon_anchor=(15, 15),  # Position the circle's center
                html=f'<div style="font-size: 10pt; text-align: center;">' +
                     f'<svg width="20" height="20">' +
                     f'<circle cx="10" cy="10" r="9" fill="green" />' +
                     f'<text x="10" y="12" fill="white" text-anchor="middle">D</text>' +
                     f'</svg>' +
                     f'</div>'
            ),
        )
        _marker.add_to(_map)
    #plot nodes
    for n in N.values():
        _text = n.ID
        _marker = folium.Marker(
            location=(n.coord.lat, n.coord.lon),  # Set the marker's latitude and longitude
            popup=folium.Popup(_text, parse_html=True),  # Set the text content inside a popup
            icon=folium.DivIcon(
                icon_size=(30, 30),  # Adjust the size of the circle as needed
                icon_anchor=(15, 15),  # Position the circle's center
                html=f'<div style="font-size: 10pt; text-align: center;">' +
                     f'<svg width="20" height="20">' +
                     f'<circle cx="10" cy="10" r="9" fill="blue" />' +
                     f'<text x="10" y="12" fill="white" text-anchor="middle">N</text>' +
                     f'</svg>' +
                     f'</div>'
            ),
        )
        _marker.add_to(_map)
    #plot routes
    for r in R.values():
        for i in range(1,len(r.exact_tour)):
            _start=r.exact_tour[i-1]
            _end= r.exact_tour[i]
            _geojson_route = route_geometry[_start][_end]
            _gj = folium.GeoJson(_geojson_route, style_function=set_style(list(mcolors.TABLEAU_COLORS.values())[1]))
            _gj.add_to(_map)

    _map.save(f'{_map_name}.html')

def gen_pop_id():
    global pop_counter
    pop_counter +=1
    _new_id = f'Ind{pop_counter}'
    return _new_id

In [8]:
class Coord:

    def __init__(self, _lon, _lat):
        self.lon = float(_lon)  # lon = x
        self.lat = float(_lat)  # lat = y

    def __str__(self):
        return f'lon:{self.lon} , lat:{self.lat}'

    def __repr__(self):
        return f'lon:{self.lon} , lat:{self.lat}'

class Route:

    def __init__(self, _id, _tour):
        self.ID = _id
        self.main_tour = _tour
        self.exact_tour = None
        self.index_main_on_exact = None
        self.demand = None
        self.length = None
        self.leaving_times = [0] * len(_tour)
        self.driving_time = None
        self.cost = 0
        self.labor_cost = 0
        self.fuel_cost = 0
        self.platoon = None

    def __str__(self):
        return f'Route {self.ID}: cost: {self.cost}, demand: {self.demand} \n ' \
               f'{self.main_tour} \n' \
               f'{self.exact_tour} \n'

    def __repr__(self):
        return f'Route {self.ID}: cost: {self.cost}, demand: {self.demand} \n ' \
               f'{self.main_tour} \n' \
               f'{self.exact_tour} \n'

    def get_id_as_num(self):
        return int(self.ID.replace('R', ''))

    def add_node(self, _node, _position):
        # position on the main tour
        self.main_tour.insert(_position, _node)  # insert node on main tour
        node_before = self.main_tour[_position - 1]
        node_after = self.main_tour[_position + 1]
        self.demand += V[_node].demand  # increase demand of the route by the demand of the newly added customer
        self.length = self.length - dist_fwa[node_before][node_after] + dist_fwa[node_before][_node] + dist_fwa[_node][node_after]

        # add path from before to new
        self.exact_tour = include_path_into_tour(self, node_before, _node)
        # add path from new to after
        self.exact_tour = include_path_into_tour(self, _node, node_after)

        # replaced by the function above, but saved just in case. spent too much time on it to just delete it
        # # remove the old path from the node before to the node after the newly added one
        # ind_before = self.index_main_on_exact[_position - 1]  # index of the main node before the newly added one
        # ind_after = self.index_main_on_exact[_position + 1]  # index of the main node after the newly added one
        # del self.exact_tour[ind_before + 1:ind_after]  # del aux nodes from before node to after node
        # ind_insert = ind_before + 1  # index on the main tour, where new nodes of
        # # include new paths, assuming the shortest path is taken
        # path_from_new_to_after = path_fwa[_node, self.main_tour[_position + 1]][1:-1]
        # path_from_new_to_after.reverse()
        # for n in path_from_new_to_after:
        #     self.main_tour.insert(ind_insert, n)
        # self.main_tour.insert(_position, _node)
        # path_from_before_to_new = path_fwa[self.main_tour[_position - 1], _node][1:-1]
        # path_from_before_to_new.reverse()
        # for n in path_from_before_to_new:
        #     self.main_tour.insert(ind_insert, n)

        # self.platoon.insert(_position, 0)
        # ToDo: finish this function,
        # toDo: how can i calculate the leaving times when im already in a platoon?
        #  Recursive function to check the route that

    def remove_node(self, _node):
        ind_node = self.main_tour.index(_node)
        node_before = self.main_tour[ind_node - 1]
        node_after = self.main_tour[ind_node + 1]
        self.main_tour.remove(_node)
        self.demand += V[_node].demand
        self.length = self.length + dist_fwa[node_before][node_after] - dist_fwa[node_before][_node] - dist_fwa[_node][node_after]
        self.exact_tour = include_path_into_tour(self, node_before, node_after)

    def update_visits(self, drive_time=None):
        # update times first to make sure they are correct
        self.update_times(drive_time)
        # clear all leaving times of this route
        for n in self.main_tour:
            Visits[n][self.ID] = []
        # add all leaving times again
        for n in range(len(self.main_tour)):
            Visits[self.main_tour[n]][self.ID].append(self.leaving_times[n])

    def update_times(self, drive_time_func=None):
        if drive_time_func is None:
            drive_time_func = drive_time_graph
        # labor cost
        if sum(self.platoon) == 0:  # if there are no platoons, it's easy to calculate the driving and leaving times
            self.driving_time = V[self.exact_tour[0]].service_time  # leaving time of first node (usually depot, so 0)
            self.leaving_times = [0] * len(self.exact_tour)
            for i in range(1, len(self.exact_tour)):
                self.driving_time += drive_time_func[self.exact_tour[i - 1]][self.exact_tour[i]] \
                                     + V[self.exact_tour[i]].service_time / Len_time_step
                self.leaving_times[i] = self.driving_time
        else:
            self.driving_time = self.leaving_times[-1]
        self.labor_cost = Cost_factor_labor * self.driving_time

    def update_demand(self):
        self.demand = 0
        for n in self.main_tour:
            self.demand += V[n].demand

    def update_route(self, distance_func=None, drive_time_func=None):
        if distance_func is None:
            distance_func = dist_graph
        # fuel cost
        self.length = 0
        for i in range(1, len(self.exact_tour)):
            self.length += round(distance_func[self.exact_tour[i - 1]][self.exact_tour[i]], 2)
        self.fuel_cost = Cost_factor_fuel * self.length
        # demand
        self.demand = 0
        for n in self.exact_tour:
            self.demand += V[n].demand
        # update leaving times
        self.update_times(drive_time_func=drive_time_func)
        # update visits
        # update total cost
        self.cost = self.fuel_cost + self.labor_cost
        self.update_visits()

    def find_exact_tour(self):
        _new_path = [self.main_tour[0]]
        for _end in range(1, len(self.main_tour)):
            # path = find_exact_path(self.main_tour[start], self.main_tour[start + 1], predecessor)
            _path = path_fwa[self.main_tour[_end - 1]][self.main_tour[_end]]
            _new_path = _new_path + _path[1:]
        self.exact_tour = _new_path
        self.update_index()
        self.update_length()

    def update_length(self, distance_func=None):
        if distance_func is None:
            distance_func = dist_graph
        # fuel cost
        self.length = 0
        for i in range(1, len(self.exact_tour)):
            self.length += round(distance_func[self.exact_tour[i - 1]][self.exact_tour[i]], 2)
        self.fuel_cost = Cost_factor_fuel * self.length

    def update_index(self):
        self.index_main_on_exact = [None] * len(self.main_tour)
        for n in range(len(self.main_tour)):
            self.index_main_on_exact[n] = self.exact_tour.index(self.main_tour[n])
        self.index_main_on_exact[-1] = len(self.exact_tour) - 1
        # depot is twice in the list, always first index is found, so second time has to be corrected

class Node:

    def __init__(self, node_id, _lon, _lat, category, q=0, st=0):
        self.ID = node_id
        self.coord = Coord(_lon=_lon, _lat=_lat)
        self.category = category
        self.demand = q
        self.service_time = st

    def __str__(self):
        return f'(Node: ID:{self.ID}, type:{self.category}, coord:{self.coord}, demand:{self.demand}, ' \
               f'service time:{self.service_time}) \n'

    def __repr__(self):
        return f'(Node: ID:{self.ID}, type:{self.category}, coord:{self.coord}, demand:{self.demand}, ' \
               f'service time:{self.service_time}) \n'

class Visit:

    def __init__(self, _route, _node, _leave_time, _successor=None, _predecessor=None, _earliest_time=0, _latest_time=None):
        if _latest_time is None:
            self.latest_time = T_max
        else:
            self.latest_time = _latest_time
        self.route = _route
        self.node = _node
        self.leave_time = _leave_time
        self.predecessor = _predecessor
        self.successor = _successor
        self.earliest_time = _earliest_time

    def __str__(self):
        return f'Node {self.node} on route {self.route} leaving at {self.leave_time} to {self.successor}.'

    def __repr__(self):
        return f'Node {self.node} on route {self.route} leaving at {self.leave_time} to {self.successor}.'

class Genecode:

    def __init__(self, gene, depot_allocation, travel_cost=None):
        self.gene = gene
        self.depot_allocation = depot_allocation
        if travel_cost is None:
            self.travel_cost = dist_fwa
        else:
            self.travel_cost = travel_cost

    def __str__(self):
        return f'Gene: {self.gene} with depots {self.depot_allocation}.'

    def __repr__(self):
        return f'Gene: {self.gene} with depots {self.depot_allocation}.'

    def recreate_tour(self):
        for d in D:
            # create list of all visited customers for each depot
            _gene_depot = []
            for c_ind in range(len(self.gene)):
                if self.depot_allocation[c_ind] == d:
                    _gene_depot.append(self.gene[c_ind])
            del c_ind

            # create network
            _network = {i: dict() for i in [d] + _gene_depot}  # depot is possible starting node

            for c1_ind in range(len(_gene_depot)):
                c1 = _gene_depot[c1_ind]
                # it´s easier to find c with the index than to find the index with c,
                # since c could be twice or more in the list
                _demand = 0
                for c2_ind in range(c1_ind, len(_gene_depot)):
                    c2 = _gene_depot[c2_ind]
                    if _demand + V[c2].demand <= Capacity_truck:
                        _demand += V[c2].demand
                        # calculate the costs of the subtour
                        _cost = 0

                        if c2_ind - c1_ind == 0:
                            # only one node in subtour
                            _cost += self.travel_cost[d][c1]
                            _cost += self.travel_cost[c1][d]

                        for n_ind in range(c1_ind, c2_ind):
                            n = _gene_depot[n_ind]
                            if n == c1:
                                # first node of subtour
                                _cost += self.travel_cost[d][n]

                            if c2_ind - c1_ind > 0:
                                # middle node
                                _cost += self.travel_cost[n][_gene_depot[n_ind + 1]]

                            if n == _gene_depot[c2_ind]:
                                # last node of subtour
                                _cost += self.travel_cost[n][d]
                        del n_ind

                    else:
                        # as soon as the demand is too high once, it will be too high for all the following nodes too
                        break

                    _network[c1][c2] = _cost
                del c2_ind
            del c1_ind
            print(_network)
        del d

In [9]:
data_path = r"C:\Users\phili\OneDrive\Uni\Masterarbeit\Data"
file_intersections = data_path + r'\Intersections.csv'
file_customers = data_path + r'\standorte_7.csv'
file_junctions = data_path + r'\junctions.geojson'
file_connections = data_path + r'\connections.csv'
file_parameters = data_path + r'\parameters.xlsx'


In [42]:
# read parameters

wb = openpyxl.load_workbook(file_parameters)

Max_working_time = wb['Others']['B'][1].value  # max working time
Max_driving_time = wb['Others']['B'][2].value  # max driving time
Fuel_saving_factor = wb['Others']['B'][3].value  # fuel saving factor
Driving_relief_factor = wb['Others']['B'][4].value  # driving relief factor
Cost_factor_labor = wb['Others']['B'][5].value  # labor cost
Service_time = wb['Others']['B'][7].value  # service time at customers in h
Velocity = wb['Others']['B'][8].value  # velocity of trucks
Cost_factor_depreciation = wb['Others']['B'][9].value  # cost of depreciation if truck used
Cost_factor_fuel = wb['Others']['B'][10].value  # fuel cost for trucks
Capacity_truck = wb['Others']['B'][11].value  # capacity of trucks
Len_time_step = wb['Others']['B'][12].value  # length of time steps in h
Num_depots = wb['Others']['B'][13].value # number of depots
Num_customers = wb['Others']['B'][14].value # number of customers
Borderline_factor = wb['Others']['B'][15].value
Population_size = wb['Others']['B'][16].value
Tournament_size = wb['Others']['B'][17].value
Tournament_probability = wb['Others']['B'][18].value
Mutation_chance = wb['Others']['B'][19].value
Survivor_probability = wb['Others']['B'][20].value
Number_generation = wb['Others']['B'][21].value

# change times from hours to timesteps
Max_working_time = Max_working_time / Len_time_step
Max_driving_time = Max_driving_time / Len_time_step
Service_time = Service_time /Len_time_step

# simple sets
T_max = wb['Others']['B'][6].value  # number of time steps
T = list(range(T_max))
K_max = wb['Others']['B'][0].value  # max number of trucks
K = {f'K{i}' for i in range(K_max)}

In [11]:
# read intersections of highways

df = pd.read_csv(file_intersections, delimiter=";")

df['Longitude'] = df['Longitude'].apply(coord_min_to_decimal)
df['Latitude'] = df['Latitude'].apply(coord_min_to_decimal)

# change dataframes to dict because i know better how to handle those...
intersections = df['ID'].to_list()
lat_list = df['Latitude'].to_list()
lon_list = df['Longitude'].to_list()
intersections = {intersections[i]: Coord(_lat=lat_list[i], _lon=lon_list[i]) for i in range(len(intersections))}

#create dictionary for the highways that are at the intersection:
highways = dict()
for i in intersections:
    highways_str = df.loc[df['ID'] == i]['Autobahnen'].to_list()
    highways_list = highways_str[0].split(',')
    highways[i] = highways_list
del i, highways_str, highways_list





In [12]:
# find connections
connections = []
for i in df.index:
    text = df['Verbindungen'][i][1:]
    end_points = text.split(',')
    for j in range(len(end_points)):
        end_points[j] = 'AK' + end_points[j]
        connections.append((df['ID'][i], end_points[j]))

# print(connections)
print(f'{int(len(connections))} connections were added.')
# check if all connections are two-sided
for i,j in connections:
    if (j,i) not in connections:
        print(f'Arc {j,i} is missing.')


234 connections were added.


In [13]:
# read customers/depot
customer_coords = dict()
depot_coords = dict()

with open(file_customers) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    row_count = 0
    for row in csv_reader:
        # first line is header
        # second line is depot
        if row_count == 0:
            headers = row
        elif row_count == 1:
            depot_coords[f'D{row[0]}'] = Coord(row[3], row[2])
        else:
            customer_coords[f'C{row[0]}'] = Coord(row[3], row[2])
        row_count +=1

print('Customers are read from file.')

Customers are read from file.


In [14]:
# read junctions for exits

gdf = gpd.read_file(file_junctions)
junc_ids = gdf['id'].to_list()
junc_ids = [s.replace('node/', '') for s in junc_ids]
print(len(junc_ids))
# find the convex hull of all intersections.
points = np.array([[i.lon, i.lat] for i in intersections.values()])
hull = ConvexHull(points)
hull_vertices = points[hull.vertices]
polygon = Polygon(hull_vertices)


junc_coords_list = gdf['geometry'].to_list()
junc_coords = dict()
for g_ind in range(len(junc_coords_list)):
    g = junc_coords_list[g_ind]
    if g.geom_type == 'Point':
        coord = g.xy
        point = Point(coord[0][0], coord[1][0])
        # check if junction is within polygon of interceptions
        if point.within(polygon):
            junc_coords[f'{junc_ids[g_ind]}'] = Coord(_lon=coord[0][0], _lat=coord[1][0])
del junc_ids, junc_coords_list, g, g_ind
print(f'{len(junc_coords)} junctions are read from file.')




3231
1462 junctions are read from file.


In [15]:
# choose random customers
chosen_customers = random.sample(list(customer_coords.keys()), Num_customers)
chosen_customers = {c: customer_coords[c] for c in chosen_customers}

# choose customers as additional depots
while len(depot_coords)<Num_depots:
    new_depot = random.choice(list(customer_coords.keys()))
    if (new_depot not in depot_coords) and (new_depot not in chosen_customers):
        depot_coords[new_depot.replace('C', 'D')] = Coord(_lon=customer_coords[new_depot].lon, _lat=customer_coords[new_depot].lat)

# add variables for heuristic
E = set()
dist_graph = {i: dict() for i in intersections}
chosen_gitter_points = {i: dict() for i in intersections}
route_geometry = {i: dict() for i in intersections}

In [16]:

gitter_points = {i: [] for i in intersections}
marker_size = 30

loc = ox.geocode('NRW, Germany')
m_test = folium.Map(location=loc, zoom_start=12)

#plot main and all gitter points
for p in intersections:
    #find gitter points
    abweichung = list(range(-3,4,1))
    factor = 0.00014
    gitter = [(factor * a, factor* b) for a in abweichung for b in abweichung]
    gitter_points[p] = [(intersections[p].lat + j[0], intersections[p].lon + j[1]) for j in gitter]

    # plot main point
    marker_text = p[2:]
    marker = folium.Marker(
        location=(intersections[p].lat, intersections[p].lon),  # Set the marker's latitude and longitude
        popup=folium.Popup(p, parse_html=True),  # Set the text content inside a popup
        icon=folium.DivIcon(
            icon_size=(30, 30),  # Adjust the size of the circle as needed
            icon_anchor=(15, 15),  # Position the circle's center
            html=f'<div style="font-size: 10pt; text-align: center;">' +
                 f'<svg width="20" height="20">' +
                 f'<circle cx="10" cy="10" r="9" fill="blue" />' +
                 f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +
                 f'</svg>' +
                 f'</div>'
        ),
    )
    marker.add_to(m_test)

    # plot the gitter
    plot_gitter = False
    if plot_gitter:
        marker_text = 0
        for loc in gitter_points[p]:
            #print(loc)
            marker = folium.Marker(
                location=loc,  # Set the marker's latitude and longitude
                popup=folium.Popup(p, parse_html=True),  # Set the text content inside a popup
                icon=folium.DivIcon(
                    icon_size=(marker_size, marker_size),  # Adjust the size of the circle as needed
                    #icon_anchor=(10, 10),  # Position the circle's center
                    html=f'<div style="font-size: 10pt; text-align: center;">' +
                         f'<svg width="20" height="20">' +
                         f'<circle cx="10" cy="10" r="9" fill="black" />' +
                         f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
                         f'</svg>' +
                         f'</div>'
                ),
            )
            marker.add_to(m_test)
            marker_text +=1
print('\n ---- All points are added to the map! ---- \n')
m_test.save('map_network.html')


 ---- All points are added to the map! ---- 



In [17]:
# add the connections
new_connections = False
if new_connections:
    list_for_writing = []
    for c in connections:
        start = c[0]
        end = c[1]

        # first find the correct autobahn
        connecting_highway = None
        for a in highways[start]:
            if a in highways[end]:
                connecting_highway = a
                break

        coordinates = gitter_points[start] + gitter_points[end]
        coordinates = [(j,i) for i,j in coordinates] # switch von lat,lon to lon, lat


        # no possible way to know which node points in which direction, so have to check all for directions
        dist_matrix = client.distance_matrix(
            locations =coordinates, # lon, lat
            profile='driving-hgv',
            sources= list(range(len(gitter_points[start]))), # start point as sources
            destinations=list(range(len(gitter_points[start]), len(coordinates) )), # only the end points are needed as destinations
            metrics=['duration'],
            units='km'
        )
        dist_matrix_km = np.array(dist_matrix['durations'])
        min_value = np.min(dist_matrix_km)
        ind_start, ind_end = np.where(dist_matrix_km == min_value)
        ind_start = int(ind_start[0])
        ind_end = int(ind_end[0])


        # plot the route
        # ATTENTION: Longitude and latitude are switched compared to plotting
        coordinates = ((gitter_points[start][ind_start][1],gitter_points[start][ind_start][0]), (gitter_points[end][ind_end][1],gitter_points[end][ind_end][0]))
        direction_params = {'coordinates': coordinates, #lon, lat
                            'profile': 'driving-hgv',
                            'format_out': 'geojson',
                            'preference': 'shortest',
                            'geometry': 'true'}
        route = directions(client=client, **direction_params)
        time.sleep(1.6)  # wait for a short time because the server only allows 40 accesses per minute
        distance, duration = route['features'][0]['properties']['summary'].values()

        # save variables for writing to file
        list_to_add = [start, end, ind_start, ind_end, distance, route]
        list_for_writing.append(list_to_add)
        print(f'Connection {start, end} added successfully!')
    del c

    with open(file_connections, 'w') as csv_file:
        csvwriter = csv.writer(csv_file)
        csvwriter.writerows(list_for_writing)
    print('New connections created and saved to file.')



In [18]:

# a file with the connections exists already
E = set()
with open(file_connections, mode='r') as file_temp:

    # reading the CSV file
    csvFile = csv.reader(file_temp)
    # displaying the contents of the CSV file
    for lines in csvFile:
        if len(lines)>0:
            start = lines[0]
            end = lines[1]
            ind_start = lines[2]
            ind_end = lines[3]
            distance = lines[4]
            geojson_string = lines[5]
            geojson_string = geojson_string.replace("'", "\"")
            geojson_string = geojson_string.replace('True', '\"True\"')
            geojson_string = geojson_string.replace('False', '\"False\"')
            geojson_route = json.loads(geojson_string)

            E.add((start, end))
            dist_graph[start][end] = float(distance)/1000
            chosen_gitter_points[start, end] = [ind_start, ind_end]
            route_geometry[start][end] = geojson_route
            gj = folium.GeoJson(geojson_route, style_function=set_style(list(mcolors.TABLEAU_COLORS.values())[0]))
            gj.add_to(m_test)


    print('Connections read from file. \nMap was updated with connections.')



m_test.save('map_network.html')

Connections read from file. 
Map was updated with connections.


In [19]:
# find the closest junctions to each customer
all_coords = dict()
all_coords.update(junc_coords)
all_coords.update(intersections)
all_coords.update(customer_coords)
all_coords.update(depot_coords)
chosen_junctions = []
for c in list(chosen_customers.keys()) + list(depot_coords.keys()):
    print(c, all_coords[c])
    # plot the customer
    marker_text = c
    marker = folium.Marker(
        location=(all_coords[c].lat, all_coords[c].lon),  # Set the marker's latitude and longitude
        popup=folium.Popup(c, parse_html=True),  # Set the text content inside a popup
        icon=folium.DivIcon(
            icon_size=(30, 30),  # Adjust the size of the circle as needed
            icon_anchor=(15, 15),  # Position the circle's center
            html=f'<div style="font-size: 10pt; text-align: center;">' +
                 f'<svg width="20" height="20">' +
                 f'<circle cx="10" cy="10" r="9" fill="red" />' +
                 f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
                 f'</svg>' +
                 f'</div>'
        ))
    marker.add_to(m_test)

    dist_to_junction = {j: pythagoras(all_coords[c], all_coords[j]) for j in junc_coords}
    closest_junction = min(dist_to_junction, key=dist_to_junction.get)
    print(closest_junction, all_coords[closest_junction])
    if not closest_junction in chosen_junctions:  #if the junction has not been added before find the highway its on
        chosen_junctions.append(closest_junction)
        # plot the junction
        marker_text = closest_junction
        marker = folium.Marker(
            location=(all_coords[closest_junction].lat, all_coords[closest_junction].lon),  # Set the marker's latitude and longitude
            popup=folium.Popup(closest_junction, parse_html=True),  # Set the text content inside a popup
            icon=folium.DivIcon(
                icon_size=(30, 30),  # Adjust the size of the circle as needed
                icon_anchor=(15, 15),  # Position the circle's center
                html=f'<div style="font-size: 10pt; text-align: center;">' +
                     f'<svg width="20" height="20">' +
                     f'<circle cx="10" cy="10" r="9" fill="black" />' +
                     f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
                     f'</svg>' +
                     f'</div>'
            ))
        marker.add_to(m_test)

        min_dist_to_highway = np.inf
        chosen_connection = None
        for start, end in connections:
            path = route_geometry[start][end]['features'][0]['geometry']['coordinates']
            distance_junc_to_geometry = [pythagoras(all_coords[closest_junction], Coord(p[0], p[1] )) for p in path]
            min_dist_junc_to_geometry = min(distance_junc_to_geometry)
            if min_dist_junc_to_geometry < min_dist_to_highway:
                min_dist_to_highway = min_dist_junc_to_geometry
                chosen_connection = (start, end)

        # divide highway

        # delete old connection
        p1 = chosen_connection[0]
        p2 = chosen_connection[1]
        # remove old connections
        E.remove((p1,p2))
        E.remove((p2,p1))
        connections.remove((p1, p2))
        connections.remove((p2, p1))
        del dist_graph[p1][p2]
        del dist_graph[p2][p1]
        del route_geometry[p1][p2]
        del route_geometry[p2][p1]
        del p1, p2

        # add new connections
        for p in chosen_connection:
            E.add((p, closest_junction))
            E.add((closest_junction, p))
            connections.append((p, closest_junction))
            connections.append((closest_junction, p))
            # find the correct connection with distance matrix
            if p in gitter_points.keys():
                coordinates = [(all_coords[closest_junction].lat, all_coords[closest_junction].lon)] + gitter_points[p]
            else:
                coordinates = [(all_coords[closest_junction].lat, all_coords[closest_junction].lon)] + [(all_coords[p].lat, all_coords[p].lon)]
            coordinates = [(j,i) for i,j in coordinates] # switch von lat,lon to lon, lat
            # no possible way to know which node points in which direction, so have to check all for directions
            dist_matrix = client.distance_matrix(
                locations =coordinates, # lon, lat
                profile='driving-hgv',
                sources= [0], # junction as source
                destinations=list(range(1, len(coordinates))), # only the end points are needed as destinations
                metrics=['duration'],
                units='km')
            dist_matrix_km = np.array(dist_matrix['durations'])
            min_value = np.min(dist_matrix_km)
            ind_start, ind_end = np.where(dist_matrix_km == min_value)
            ind_start = int(ind_start[0])
            ind_end = int(ind_end[0])
            if p in gitter_points.keys():
                route_temp, dist_temp = distance_ors(_start_lon=all_coords[closest_junction].lon,
                                                     _start_lat=all_coords[closest_junction].lat,
                                                     _end_lon=gitter_points[p][ind_end][1],
                                                     _end_lat=gitter_points[p][ind_end][0])
            else:
                route_temp, dist_temp = distance_ors(_start_lon=all_coords[closest_junction].lon,
                                                     _start_lat=all_coords[closest_junction].lat,
                                                     _end_lon=all_coords[p].lon,
                                                     _end_lat=all_coords[p].lat)


            dist_graph[p][closest_junction] = dist_temp/1000
            if not closest_junction in dist_graph.keys():
                dist_graph[closest_junction] = dict()
            dist_graph[closest_junction][p] = dist_temp/1000
            route_geometry[p][closest_junction] = route_temp
            if not closest_junction in route_geometry.keys():
                route_geometry[closest_junction] = dict()
            route_geometry[closest_junction][p] = route_temp
            # it´s okay that the route here is the wrong way, it´s mainly for plotting. But needs to be there, in case of another junctions that divides it.
        del p
        # add new connections for second node
        '''E.add((p2, closest_junction))
        E.add((closest_junction, p2))
        connections.append((p2, closest_junction))
        connections.append((closest_junction, p2))
        route_temp, dist_temp = distance_ors(start_lat=all_coords[p2].lat,
                                             start_lon=all_coords[p2].lon,
                                             end_lon=all_coords[closest_junction].lon,
                                             end_lat=all_coords[closest_junction].lat
                                             )
        dist_graph[p2][closest_junction] = dist_temp
        if not closest_junction in dist_graph.keys():
            dist_graph[closest_junction] = dict()
        dist_graph[closest_junction][p2] = dist_temp
        route_geometry[p2][closest_junction] = route_temp
        if not closest_junction in route_geometry.keys():
            route_geometry[closest_junction] = dict()
        route_geometry[closest_junction][p2] = route_temp'''
        # it´s okay that the route here is the wrong way, it´s mainly for plotting. But needs to be there, in case of another junctions that divides it.


    # add edge (customer, junction)
    E.add((c, closest_junction))
    E.add((closest_junction, c))
    #connections.append((c, closest_junction))
    #connections.append((closest_junction, c))
    # dont add edges with customers to connections. For some reason it happens that this is found for the nearest junction.
    route_temp1, dist_temp1 = distance_ors(_start_lon=all_coords[c].lon,
                                           _start_lat=all_coords[c].lat,
                                           _end_lon=all_coords[closest_junction].lon,
                                           _end_lat=all_coords[closest_junction].lat
                                           )
    route_temp2, dist_temp2 = distance_ors(_end_lon=all_coords[c].lon,
                                           _end_lat=all_coords[c].lat,
                                           _start_lon=all_coords[closest_junction].lon,
                                           _start_lat=all_coords[closest_junction].lat
                                           )
    print(dist_temp1, dist_temp2)
    # check which direction is shorter, since the junction is on the highway, it might be the wrong direction and then there is a huge detour
    if dist_temp1 < dist_temp2 :
        p1 = c
        p2 = closest_junction
        dist_temp = dist_temp1
        route_temp = route_temp1
    else:
        p1 = closest_junction
        p2 = c
        dist_temp = dist_temp2
        route_temp = route_temp2
    # adding distance
    if not p1 in dist_graph.keys():
        dist_graph[p1] = dict()
    dist_graph[p1][p2] = dist_temp/1000
    if not p2 in dist_graph.keys():
        dist_graph[p2] = dict()
    dist_graph[p2][p1] = dist_temp/1000

    if not p1 in route_geometry.keys():
        route_geometry[p1] = dict()
    route_geometry[p1][p2] = route_temp
    if not p2 in route_geometry.keys():
        route_geometry[p2] = dict()
    route_geometry[p2][p1] = route_temp
    gj = folium.GeoJson(route_temp, style_function=set_style(list(mcolors.TABLEAU_COLORS.values())[0]))
    gj.add_to(m_test)

print('All customers have found their junctions.')
m_test.save('map_network.html')





C5054 lon:6.5762593 , lat:50.5441357
300693337 lon:6.6707102 , lat:50.5283228
24833.4 12110.9
C4212 lon:6.99829 , lat:51.16306
251368385 lon:6.9647906 , lat:51.1553517
9954.8 12655.9
C4014 lon:6.7621643 , lat:51.4021407
25734257 lon:6.7723422 , lat:51.4032074
5333.6 1235.9
C5111 lon:7.5678347 , lat:51.0249098
26151800 lon:7.5459005 , lat:50.9837194
8027.2 5375.7
C5493 lon:6.9260209 , lat:51.5201334
642263 lon:6.9418279 , lat:51.5066148
10273.6 2448.2
C5335 lon:7.0317611 , lat:51.4505741
160719 lon:7.0334104 , lat:51.4381981
7480.0 1748.8
C5805 lon:6.8334581 , lat:51.1756642
38578733 lon:6.8362217 , lat:51.1691357
2448.8 1450.4
C5257 lon:7.026613 , lat:50.8301037
91447577 lon:6.9958334 , lat:50.8055278
16647.3 17727.5
C4379 lon:6.8348702 , lat:51.203631
253869269 lon:6.8382438 , lat:51.190023
5252.2 4565.1
C4624 lon:7.0425985 , lat:51.3412349
68753204 lon:7.0424069 , lat:51.3502265
3541.6 2965.4
C4963 lon:7.1971431 , lat:51.271011
234155 lon:7.2036187 , lat:51.289404
3847.4 3302.1
C5601

In [20]:
# create sets for algorithm
N = {n: Node(node_id=n,_lon=junc_coords[n].lon, _lat=junc_coords[n].lat, category='auxiliary' ) for n in chosen_junctions}
N.update({n: Node(node_id=n, _lon=intersections[n].lon, _lat=intersections[n].lat, category='auxiliary') for n in intersections})
C = {c: Node(node_id=c, _lon=customer_coords[c].lon, _lat=customer_coords[c].lat, category='customer', st=Service_time, q=generate_demand()) for c in chosen_customers}
D = {d: Node(node_id=d, _lon=depot_coords[d].lon, _lat=depot_coords[d].lat, category='depot') for d in depot_coords}

V=dict()
V.update(N)
V.update(C)
V.update(D)

Visits = {v: dict() for v in V}
Platoon_arcs = {i: {j: [] for j in dist_graph[i]} for i in dist_graph}
R = dict() # routes
Visits_dict = {r_ind: [] for r_ind in R}
R_at_depot = {d: [] for d in D} # which route starts at which depot



In [21]:
dist_fwa = {i: {j: dist_graph[i][j] if (i, j) in E else np.inf for j in V} for i in V}
pred_fwa = {i: {j: i if (i, j) in E else None for j in V} for i in V}
path_fwa = {i: {j: [i, j] if (i, j) in E else None for j in V} for i in V}
path_alt = {i: {j: [] for j in V} for i in V}  # list of alternative path
drive_time_graph = calc_driving_time(dist_graph)

# find fastest and alternative tours from any customer/depot to all other customer/depot
detour_factor = (Cost_factor_fuel + Cost_factor_labor / Velocity) / ((1 - Fuel_saving_factor) * Cost_factor_fuel + Cost_factor_labor / Velocity)

# floyd warshall algorithm, incl. the path and also alternatives
dist_fwa = {i: {j: dist_graph[i][j] if (i, j) in E else np.inf for j in V} for i in V}
pred_fwa = {i: {j: i if (i, j) in E else None for j in V} for i in V}
path_fwa = {i: {j: [i, j] if (i, j) in E else None for j in V} for i in V}
path_alt = {i: {j: [] for j in V} for i in V}  # list of alternative path

for i in V:
    dist_fwa[i][i] = 0
    pred_fwa[i][i] = i
    path_fwa[i][i] = [i]
del i

for k in V:
    for i in V:
        for j in V:
            if dist_fwa[i][j] > dist_fwa[i][k] + dist_fwa[k][j]:
                dist_fwa[i][j] = round(dist_fwa[i][k] + dist_fwa[k][j], 2)
                pred_fwa[i][j] = pred_fwa[k][j]
                path_fwa[i][j] = path_fwa[i][k] + path_fwa[k][j][1:]
            elif dist_fwa[i][j] * detour_factor > dist_fwa[i][k] + dist_fwa[k][j]:
                path_alt[i][j].append((path_fwa[i][k] + path_fwa[k][j][1:], dist_fwa[i][k] + dist_fwa[k][j]))
                # store in a tuple to easier access the distance of the path
        del j
    del i
del k
#remove all alternative paths, that are now too long by creating a new list
for i in V:
    for j in V:
        if (i in C or i in D) and (j in C or j in D):
            path_alt[i][j] = [alt_path for alt_path in path_alt[i][j]
                          if alt_path[1] < dist_fwa[i][j] * detour_factor/2]
del i, j

drive_time_fwa = calc_driving_time(dist_fwa)

In [34]:
allocation = {d: [] for d in D}
alternative_depot = dict()
for c in C:
    dist_depot = {d: dist_fwa[c][d] for d in D}
    d = min(dist_depot, key=dist_depot.get)
    allocation[d].append(c)
    alternative_depot[c] = find_borderline_alternative_depot(c, d)
del c, dist_depot
print(allocation)
demand_depot = {d: sum([V[i].demand for i in allocation[d]]) for d in D}
# create initial population for GA
population = dict()
pop_counter = 0
while len(population) < Population_size:
    route_list = []
    # create random routes
    for d in allocation:
        cust_at_depot = [c for c in allocation[d]]  # copy so original allocation isn't changed
        route = [d, d]
        route_demand = 0
        route_drivetime = 0
        route_worktime = 0
        while len(cust_at_depot) >0:
            next_customer = random.choice(cust_at_depot)
            cust_at_depot.remove(next_customer)
            if check_route_conditions(route, next_customer, route_demand, route_drivetime, route_worktime):
                # insert new customer to route and update route parameter
                route_demand += V[next_customer].demand
                diff_drivetime = drive_time_fwa[route[-2]][next_customer] + drive_time_fwa[next_customer][route[-1]] - drive_time_fwa[route[-2]][route[-1]]
                route_drivetime +=  diff_drivetime
                route_worktime = route_worktime + diff_drivetime + V[next_customer].service_time
                route.insert(-1, next_customer)
                if len(cust_at_depot) == 0:
                    route_list.append(route)
            else:
                # start new route
                route_list.append(route) # save finished route
                route = [d, next_customer, d]
                route_demand = V[next_customer].demand
                route_drivetime = drive_time_fwa[d][next_customer] + drive_time_fwa[next_customer][d]
                route_worktime  = route_drivetime + V[next_customer].service_time

    population[gen_pop_id()] = route_list

for p in population:
    print(population[p])
# evaluate fitness
fitness = dict()
for individual in population:
    fitness[individual] = evaluate_fitness_of_individual(population[individual])
print(fitness)

chromosomes = dict()
for key, individual in population.items():
    chromosomes[key]=create_chromosome(individual)

for c in chromosomes:
    print(c, chromosomes[c])

{'D1': ['C5111', 'C4963', 'C5601', 'C4012', 'C5620'], 'D4021': ['C5054', 'C4212', 'C4014', 'C5493', 'C5335', 'C5805', 'C5257', 'C4379', 'C4624', 'C4769']}
[['D1', 'C5620', 'C5601', 'C4963', 'C5111', 'C4012', 'D1'], ['D4021', 'C5257', 'C5335', 'C5054', 'C5805', 'C4379', 'D4021'], ['D4021', 'C5493', 'C4014', 'C4769', 'C4212', 'C4624', 'D4021']]
[['D1', 'C5601', 'C5620', 'C4963', 'C5111', 'C4012', 'D1'], ['D4021', 'C5805', 'C4379', 'C4769', 'C4624', 'C4212', 'C5257', 'D4021'], ['D4021', 'C5335', 'C4014', 'C5054', 'C5493', 'D4021']]
[['D1', 'C5111', 'C4012', 'C5601', 'C4963', 'C5620', 'D1'], ['D4021', 'C5805', 'C4624', 'C5054', 'C5257', 'C4379', 'D4021'], ['D4021', 'C5335', 'C4014', 'C5493', 'C4212', 'C4769', 'D4021']]
[['D1', 'C5111', 'C5620', 'C4963', 'C4012', 'C5601', 'D1'], ['D4021', 'C5335', 'C5054', 'C4212', 'C5257', 'C4379', 'D4021'], ['D4021', 'C5805', 'C4624', 'C4769', 'C4014', 'C5493', 'D4021']]
[['D1', 'C5620', 'C5111', 'C4012', 'C5601', 'C4963', 'D1'], ['D4021', 'C4212', 'C4379

In [71]:
generation_counter = 0
while generation_counter < Number_generation:
    # parental selection
    parent1, parent2 = tournament_selection(chromosomes, fitness)


    # crossover operator
    operator = random.randint(1,2)
    if operator==1:
        child = crossover_order(chromosomes[parent1], chromosomes[parent2])
    elif operator == 2:
        child = crossover_alternating_edges(chromosomes[parent1], chromosomes[parent2])


    # mutation operator
    mutation_operator = random.randint(1,3)

    if mutation_operator == 1:
        for d in child:  # each depot gene has a chance to be mutated
            mutation_random = random.random()
            if mutation_random <= Mutation_chance:
                child[d]  = mutation_swap(child[d])
    if mutation_operator == 2:
        for d in child:  # each depot gene has a chance to be mutated
            mutation_random = random.random()
            if mutation_random <= Mutation_chance:
                child[d]  = mutation_inversion(child[d])
    if mutation_operator == 3:
        mutation_random = random.random()
        if mutation_random <= Mutation_chance:
            child  = mutation_depot_switch(child)

    # add child to population
    child_id = gen_pop_id()
    population[child_id] = route_scheduler(child)
    fitness[child_id] = evaluate_fitness_of_individual(population[child_id])
    # survivor selection
    population = survivor_selection(population)

    # increase generation
    generation_counter +=1
    print(generation_counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [72]:
print(fitness)

{'Ind1': 1355.858, 'Ind2': 1373.83, 'Ind3': 1364.112, 'Ind4': 1535.032, 'Ind5': 1401.214, 'Ind6': 1365.0520000000001, 'Ind7': 1426.9060000000002, 'Ind8': 1384.3200000000002, 'Ind9': 1527.9399999999998, 'Ind10': 1408.2, 'Ind11': 1438.168, 'Ind18': 300, 'Ind19': 300, 'Ind20': 300, 'Ind21': 300, 'Ind22': 300, 'Ind23': 300, 'Ind24': 300, 'Ind25': 300, 'Ind26': 300, 'Ind27': 300, 'Ind28': 300, 'Ind29': 300, 'Ind30': 300, 'Ind31': 300, 'Ind32': 300, 'Ind33': 300, 'Ind34': 300, 'Ind35': 300, 'Ind36': 300, 'Ind37': 300, 'Ind38': 300, 'Ind39': 300, 'Ind40': 300, 'Ind41': 300, 'Ind42': 300, 'Ind43': 300, 'Ind44': 300, 'Ind45': 300, 'Ind46': 300, 'Ind47': 300, 'Ind48': 300, 'Ind49': 300, 'Ind50': 300, 'Ind51': 300, 'Ind52': 300, 'Ind53': 300, 'Ind54': 300, 'Ind55': 300, 'Ind56': 300, 'Ind57': 300, 'Ind58': 300, 'Ind59': 300, 'Ind60': 300, 'Ind61': 300, 'Ind62': 300, 'Ind63': 300, 'Ind64': 300, 'Ind65': 300, 'Ind66': 300, 'Ind67': 300, 'Ind68': 300, 'Ind69': 300, 'Ind70': 300, 'Ind71': 300, 'Ind72

In [52]:
print(chromosomes[parent1])
print(chromosomes[parent2])


{'D1': ['C5111', 'C4012', 'C5601', 'C4963', 'C5620'], 'D4021': ['C5805', 'C4624', 'C5054', 'C5257', 'C4379', 'C5335', 'C4014', 'C5493', 'C4212', 'C4769']}
{'D1': ['C5620', 'C5601', 'C4963', 'C5111', 'C4012'], 'D4021': ['C5257', 'C5335', 'C5054', 'C5805', 'C4379', 'C5493', 'C4014', 'C4769', 'C4212', 'C4624']}


In [ ]:
for chrom in chromosomes:
    population[gen_pop_id()] = route_scheduler(chrom)

In [ ]:
alternative_depot

In [ ]:
# ---create initial routes with a savings heuristic--- #
initial_routes = []
max_route_ID = 0
for i in Visits:  # reset dict visits
    Visits[i] = dict()
del i
route_visited = dict()  # local dict to see on which route the nodes are, only works for the initial routes because nodes can be on several routes due to split delivery
list_route_keys = list(R.keys())
for k in list_route_keys:
    del R[k]
del list_route_keys

# create initial routes for each depot separately
for d in D:
    if len(initial_routes) > 0:
        max_route_ID = max([int(r.ID.replace('R','')) for r in initial_routes]) + 1
    # create a route for each customer
    # clear route dict, just in case if the cell is run a second time
    routes_depot = dict()
    for i in range(len(allocation[d])):
        route_id = f'R{(i +max_route_ID)}'
        routes_depot[route_id] = Route(_id=route_id, _tour=[d, allocation[d][i], d])
        route_visited[allocation[d][i]] = route_id



    #routes = {f'R{(i +max_route_ID)}': Route(_id=f'R{(i +max_route_ID)}', _nodes=[d, allocation[d][i], d]) for i in range(len(allocation[d]))} # routes at a depot, to be able to delete it later with the key
    for _, r in routes_depot.items():
        r.update_demand()

    nodes_at_depot = [d] + allocation[d].copy() # the depot and all customers allocated to it
    costs = {i: {j: dist_fwa[i][j] for j in nodes_at_depot if j!= i } for i in nodes_at_depot}  # all customers and the depot itself at this depot
    savings = {(i, j): (round(costs[i][d] + costs[d][j] - costs[i][j],2)) for j in allocation[d]  for i in allocation[d] if i != j}
    while len(savings) > 0:
        # find the biggest saving
        (n1,n2) = max(savings, key=savings.get)

        # update routes
        r1 = route_visited[n1]
        r2 = route_visited[n2]
        # r1 = list(visits[n1].keys())[0]
        # r2 = list(visits[n2].keys())[0]
        if routes_depot[r1].demand + routes_depot[r2].demand < Capacity_truck:
            pos1 = routes_depot[r1].main_tour.index(n1)
            pos2 = routes_depot[r2].main_tour.index(n2)
            # XOR operator, if NOT exactly one of the positions is 1, the second route has to be reversed
            if not ((pos1==1) ^ (pos2==1)):
                routes_depot[r2].main_tour.reverse()
            routes_depot[r1].main_tour =routes_depot[r1].main_tour[0:-1] + routes_depot[r2].main_tour[1:] # new combined route

            for n in routes_depot[r2].main_tour[0:-1]: # change the route of all nodes in r2 to r1
                route_visited[n] = r1
            del routes_depot[r2] # delete old route

            routes_depot[r1].update_demand()  # update demand of current node

            # update costs with the new routes
            costs[d][n2] = costs[d][n1] + costs[n1][n2]
            costs[n1][d] = costs[n1][n2] + costs[n2][d]

            # recalculate savings
            savings = dict()
            for r1 in routes_depot.values():
                #start and end node on current_tour
                current_nodes = (r1.main_tour[1], r1.main_tour[-2])
                for n1 in current_nodes:
                    # find all start or end notes on the other routes from this depot
                    other_nodes = set()
                    for r2 in routes_depot.values():
                        if r1.ID != r2.ID:
                            other_nodes.add(r2.main_tour[1]) # add first customer on route to nodes with possible savings
                            other_nodes.add(r2.main_tour[-2]) # add last customer on route to nodes with possible savings
                    # calculate possible savings
                    for n2 in other_nodes:
                        savings[n1, n2] = round(costs[n1][d] + costs[d][n2] - costs[n1][n2], 2)

                del other_nodes
        else: # combined demand would be over the capacity of a truck, so not possible --> delete saving
            del savings[n1,n2]

    # no more savings in for this depot possible, save routes to final routes
    initial_routes = initial_routes + list(routes_depot.values())

# cant do dict comprehension, because it would create a new dict instead of changing the existing one, which is initialized in methods
for r in initial_routes:
    R[r.ID] = r
del initial_routes, r


for r in R.values():
    r.find_exact_tour()
del r

print(R)

In [ ]:
# create visits for the initial tours
for r in R:
    Visits_dict[r] = []
for r in R.values():
    cur_time = 0
    r.leaving_times = [None] * len(r.exact_tour)
    for v_ind in range(len(r.exact_tour)):
        cur_node = r.exact_tour[v_ind]
        cur_time += V[r.exact_tour[v_ind]].service_time  # update current time with service time of current node
        r.leaving_times[v_ind] = cur_time
        if v_ind == 0: # first node in tour
            cur_predecessor = None  # doesnt have a predecessor
        else:
            cur_predecessor = r.exact_tour[v_ind-1]
        if v_ind == len(r.exact_tour) - 1:  # last node in tour
            cur_successor = None  # doesnt have a successor
        else:
            cur_successor = r.exact_tour[v_ind +1]
        Visits_dict[r.ID].append(Visit(route=r.ID, node=cur_node, leave_time=cur_time, earliest_time=cur_time, predecessor=cur_predecessor, successor=cur_successor))
        if cur_successor is not None:  # if not last node in tour
            cur_time += drive_time_graph[cur_node][cur_successor]

    del v_ind, cur_node, cur_time, cur_predecessor, cur_successor

    # calculate the latest times backwards
    cur_time = Visits_dict[r.ID][-1].latest_time
    for v_ind in range(len(Visits_dict[r.ID]) -1, -1 , -1):
        Visits_dict[r.ID][v_ind].latest_time = cur_time
        if v_ind > 0:  # if it's not the first node on the tour update cur_time with
            cur_time += - V[r.exact_tour[v_ind]].service_time  # service time
            cur_time += - drive_time_graph[r.exact_tour[v_ind - 1]][r.exact_tour[v_ind]]  # driving time
    del v_ind
del r

print('Visits calculated')

In [ ]:
plot_routes(R,'initial_routes2')

In [ ]:
for (i, j) in E:  # reset possible platoons
    Platoon_arcs[i][j] = []
for r in R.values():
    for i in range(len(r.exact_tour) - 1):
        Platoon_arcs[r.exact_tour[i]][r.exact_tour[i + 1]].append(r.ID)

for start, end_val in Platoon_arcs.items():
    for end, route_ids in end_val.items():
        # for (start,end), route_ids in platoon_arcs.items():
        if len(route_ids) > 1:
            print(start, end, route_ids)

In [ ]:
route ='R8'
start= 'AK30'
end = 'AK24'
print(R[route])
ind = R[route].exact_tour.index(start)
print(ind)
next_point = R[route].exact_tour[ind + 1]
print(next_point)
while next_point is not R[route].exact_tour[ind + 1]:
    ind = R[route].exact_tour.index(start, ind + 1)
print(ind)

#start_ind = find_start_index_of_arc(route, start, end)

In [ ]:
Platoon_arcs

In [ ]:
print('start platooning')
create_platoons()
print('First platoons created!')

In [ ]:
# plot all junctions (just for testing)
loc = ox.geocode('NRW, Germany')
map_junctions = folium.Map(location=loc, zoom_start=12)
for j in junc_coords:
    print(junc_coords[j])
    lat = junc_coords[j].lat
    lon = junc_coords[j].lon
    marker_text = j
    marker = folium.Marker(
        location=(lat, lon),  # Set the marker's latitude and longitude
        popup=folium.Popup(j, parse_html=True),  # Set the text content inside a popup
        icon=folium.DivIcon(
            icon_size=(30, 30),  # Adjust the size of the circle as needed
            icon_anchor=(15, 15),  # Position the circle's center
            html=f'<div style="font-size: 10pt; text-align: center;">' +
                 f'<svg width="20" height="20">' +
                 f'<circle cx="10" cy="10" r="9" fill="blue" />' +
                 f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
                 f'</svg>' +
                 f'</div>'
          ),
        )
    marker.add_to(map_junctions)

map_junctions.save('map_junctions.html')

In [ ]:
### ---cell not needed anymore --- ###
# create a map for all points to find the subpoints
'''
i = 31
loc = [df['Latitude'][i], df['Longitude'][i]]
m = folium.Map(location=loc, zoom_start=21)

marker_text = df['ID'][i]
marker_text = marker_text[2:]
marker_size = 30
marker = folium.Marker(
    location=(df['Latitude'][i], df['Longitude'][i]),  # Set the marker's latitude and longitude
    popup=folium.Popup(df['ID'][i], parse_html=True),  # Set the text content inside a popup
    icon=folium.DivIcon(
        icon_size=(marker_size, marker_size),  # Adjust the size of the circle as needed
        icon_anchor=(marker_size/2, marker_size/2),  # Position the circle's center
        html=f'<div style="font-size: 10pt; text-align: center;">' +
             f'<svg width="{marker_size}" height="{marker_size}">' +
             f'<circle cx="10" cy="10" r="9" fill="blue" />' +
             f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
             f'</svg>' +
             f'</div>'
    ),
    tooltip=folium.map.Tooltip("<p><b>{}</b></p><p>{}</p>".format(df['ID'][i],df['Autobahnen'][i]))
)
marker.add_to(m)

abweichung = list(range(-4,5,1))
factor = 0.00012
gitter = [(factor * a, factor* b) for a in abweichung for b in abweichung]
gitter_points = [(df['Latitude'][i] + j[0], df['Longitude'][i] + j[1]) for j in gitter]

for j in range(len(gitter)):
    marker_text = j
    marker = folium.Marker(
        location=(gitter_points[j]),  # Set the marker's latitude and longitude
        popup=folium.Popup(df['ID'][i], parse_html=True),  # Set the text content inside a popup
        icon=folium.DivIcon(
            icon_size=(marker_size, marker_size),  # Adjust the size of the circle as needed
            #icon_anchor=(10, 10),  # Position the circle's center
            html=f'<div style="font-size: 10pt; text-align: center;">' +
                 f'<svg width="20" height="20">' +
                 f'<circle cx="10" cy="10" r="9" fill="black" />' +
                 f'<text x="10" y="12" fill="white" text-anchor="middle">{marker_text}</text>' +  # Variable here
                 f'</svg>' +
                 f'</div>'
        ),
        tooltip=folium.map.Tooltip("<p><b>{}</b></p><p>{}</p>".format(df['ID'][i],df['Autobahnen'][i]))
    )
    marker.add_to(m)

m.save('stupid_map.html')
'''